# Pipeline Notebook Run

In [ ]:
from astropy.io import fits, ascii
from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
from jwst.pipeline.calwebb_spec2 import Spec2Pipeline
import sys
import os
import glob
import numpy as np
import warnings

<div class='alert alert-info'>
    <font size='3'><b>These cells contain instructions. Edit the paths below.</b></font>
</div>

In [ ]:
file_path = 'ex_data/*uncal.fits'
out_path = 'ex_proc_data/'

Find the files

In [ ]:
uncal_list = glob.glob(file_path)
n_uncal = len(uncal_list)
print(n_uncal)

Make sure NOUTPUTS exists. If not, set it to 4

In [ ]:
for one_file in uncal_list:
    head = fits.getheader(one_file)
    if 'NOUTPUTS' in head:
        pass
    else:
        print("Missing NOUTPUTS for {}, setting to 4".format(one_file))
        with fits.open(one_file,mode='update') as HDUList:
            HDUList[0].header['NOUTPUTS'] = (4, 'number of output amplifiers')

Set up stage 1

In [ ]:
pipeline = Detector1Pipeline()


# Default is to skip the persistence and IPC correction
# Make that explicit here
pipeline.persistence.skip = True
pipeline.ipc.skip = True

# Specify that you want results saved to a file
pipeline.save_results = True


if os.path.exists(out_path) == False:
    os.mkdir(out_path)
pipeline.output_dir = out_path


<div class='alert alert-info'>
    <font size='3'><b>Change the rejection threshold if needed.</b></font>
</div>

In [ ]:
pipeline.jump.rejection_threshold = 15

Run stage 1

In [ ]:
for one_file in uncal_list:
    pipeline.run(one_file)

Make sure we have rateints

In [ ]:
rateints_list = glob.glob(os.path.join(out_path,'*rateints.fits'))
n_rateints = len(rateints_list)
if n_rateints != n_uncal:
    warnings.warn("{} rateints but {} uncal".format(n_rateints,n_uncal))

Run stage 2

In [ ]:
result2 = Spec2Pipeline()
result2.save_results = True
result2.output_dir = out_path

for one_file in rateints_list:
    result2.run(one_file)
